In [155]:
import re 
import tweepy 
from tweepy import OAuthHandler  
import matplotlib.pyplot as plt
import json
import geopy
from geopy.geocoders import Nominatim   

In [47]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''
auth = OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)    
api = tweepy.API(auth)
api.verify_credentials()
print("Authentication OK")

Authentication OK


In [54]:
class Client_Twitter(object):
  
    def __init__(self): 
            
            
            # keys and tokens from the Twitter Dev Console",
        consumer_key = ''
        consumer_secret = ''
        access_token = ''
        access_token_secret = ''

           
        try: 
                 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            self.auth.set_access_token(access_token, access_token_secret)    
            self.api = tweepy.API(self.auth)
            api.verify_credentials()
            print("Authentication OK")
        except: 
            print("Error: Authentication Failed") 
                
            
    def get_tweets(self, query, count = 100): 
       
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
         
            fetched_tweets = self.api.search(q = query, count = count)
            #(fetched_tweets[0])
            
            for tweet in fetched_tweets: 
                ttext = json.dumps(tweet._json)
                parsed = json.loads(ttext)
                tweets.append(parsed)
                
               #parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e))
    
    
class Location(object):
    def getLocation(self,address):
        geolocator = Nominatim(user_agent="apa")
        location = geolocator.geocode(address)
        if location !=None:
            return([location.latitude, location.longitude])
        else:
            return("")

In [49]:
api = Client_Twitter()
loc = Location()

Authentication OK


In [50]:
tweets = api.get_tweets(query = '#coronavirus', count = 100)

In [66]:
str_list = [t["user"]["location"] for t in tweets]

In [67]:
str_list =filter(None, str_list)
str_list = list(filter(None, str_list))

In [132]:
import time
import re
from geopy.exc import GeocoderTimedOut
def do_geocode(t):
    try:
        geolocator = Nominatim(user_agent="apa")
        location = geolocator.geocode(t)
        if location !=None:
            return([location.latitude, location.longitude])
    except GeocoderTimedOut:
        return do_geocode(t)

def do_geocode_rev(latlong):
    try:
        geolocator = Nominatim(user_agent="apa")
        location = geolocator.reverse(latlong)
        if location !=None:
            #newloc= re.sub("[^a-zA-Z]+", "", location.address.split(",")[-1])
            return(location.address.split(",")[-1])
    except GeocoderTimedOut:
        return do_geocode_rev(latlong)


In [130]:
doc={}
test_dict={}
#test time
tic = time.clock()
for i,t in enumerate(str_list):
    vari = do_geocode(t)
    if(vari):
        #test
        country = do_geocode_rev("{}, {}".format(vari[0],vari[1]))
        test_dict['location{}'.format(i)]={"lat":vari[0],"lon":vari[1], "con": country}
        doc['location']={"lat":vari[0],"lon":vari[1], "con": country}
toc = time.clock()
print(toc-tic)

69.08583429999999


In [156]:
json = json.dumps(test_dict)
f = open("test_data.json","w")
f.write(json)
f.close()